In [ ]:
!sudo apt-get install -y espeak
!pip install soundfile

In [ ]:
!git clone https://github.com/flych3r/TTS.git -b pt-br
%cd TTS

!pip install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install -e .
!pip install -e .

In [ ]:
import os
import torch
import time
import IPython
import numpy as np

from TTS.tts.utils.generic_utils import setup_model
from TTS.vocoder.utils.generic_utils import setup_generator
from TTS.utils.io import load_config
from TTS.tts.utils.text.symbols import symbols, phonemes
from TTS.utils.audio import AudioProcessor
from TTS.tts.utils.synthesis import synthesis

In [ ]:
def tts(model, vocoder_model, text, CONFIG, use_cuda, ap, use_gl, trim, figures=True):
    t_1 = time.time()
    waveform, alignment, mel_spec, mel_postnet_spec, stop_tokens, inputs = synthesis(
        model, text, CONFIG, use_cuda, ap, speaker_id, style_wav=None,
        truncated=False, enable_eos_bos_chars=CONFIG.enable_eos_bos_chars,
        use_griffin_lim=use_gl, do_trim_silence=trim
    )
    # mel_postnet_spec = ap._denormalize(mel_postnet_spec.T)
    if not use_gl:
        waveform = vocoder_model.inference(torch.FloatTensor(mel_postnet_spec.T).unsqueeze(0))
        waveform = waveform.flatten()
    if use_cuda:
        waveform = waveform.cpu()
    if not isinstance(waveform, np.ndarray):
        waveform = waveform.numpy()
    rtf = (time.time() - t_1) / (len(waveform) / ap.sample_rate)
    tps = (time.time() - t_1) / len(waveform)
    print(waveform.shape)
    print(" > Run-time: {}".format(time.time() - t_1))
    print(" > Real-time factor: {}".format(rtf))
    print(" > Time per step: {}".format(tps))
    IPython.display.display(IPython.display.Audio(waveform, rate=CONFIG.audio['sample_rate']))  
    return alignment, mel_postnet_spec, stop_tokens, waveform

In [ ]:
use_cuda = False

path = '/content/drive/MyDrive/tts/'
tts_model = 'ljspeech-ddc-bn/'
TTS_MODEL = path + tts_model + 'best_model.pth.tar'
TTS_CONFIG = path + 'config.json'

vocoder_model = 'mel-gan/'
VOCODER_MODEL = path + vocoder_model + 'best_model.pth.tar'
VOCODER_CONFIG = path + 'vocoder_config.json'

# load configs
TTS_CONFIG = load_config(TTS_CONFIG)
VOCODER_CONFIG = load_config(VOCODER_CONFIG)

In [ ]:
# load the audio processor
TTS_CONFIG.audio['stats_path'] = path + 'scale_stats.npy'
ap = AudioProcessor(**TTS_CONFIG.audio)

In [ ]:
# multi speaker 
speaker_id = None
speakers = []

# load the model
num_chars = len(phonemes) if TTS_CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, len(speakers), TTS_CONFIG)

# load model state
cp =  torch.load(TTS_MODEL, map_location=torch.device('cpu'))

# load the model
model.load_state_dict(cp['model'])
if use_cuda:
    model.cuda()

# set model stepsize
if 'r' in cp:
    model.decoder.set_r(cp['r'])

model.eval();

In [ ]:
# load vocoder model
vocoder_model = setup_generator(VOCODER_CONFIG)
vocoder_model.load_state_dict(torch.load(VOCODER_MODEL, map_location="cpu")["model"])
vocoder_model.remove_weight_norm()
vocoder_model.inference_padding = 0

ap_vocoder = AudioProcessor(**VOCODER_CONFIG['audio'])
if use_cuda:
    vocoder_model.cuda()
vocoder_model.eval()

In [ ]:
sentences =  [
    "Confira as ações que o Governo do Ceará vem promovendo no combate à pandemia.",
    "Hospital Leonardo Da Vinci realiza mais de 1200 cirurgias eletivas em cerca de dois meses.",
    "Investimentos para o ensino remoto geram legado para a educação cearense em 2020.",
    "Os resultados positivos foram apresentados nesta segunda-feira (21) O Centro Socioeducativo Patativa do Assaré realizou na manhã desta segunda-feira (21) o encerramento do projeto-piloto em parceria com a Terre Des Hommes (Tdh).",
    "Ceará Transparente e IntegraSUS dão transparência as ações do Governo do Ceará no combate à Covid-19",
    "Com apoio da Funcap, pesquisadores publicam artigo sobre reprodução do sapinho maranguapense em periódico internacional",
    "Dragão do Mar celebra período natalino com espetáculo virtual de pastoril, nesta quarta-feira (23)",
    "Natal do Governo do Ceará acolhe 424 crianças e adolescentes de 19 entidades",
    "CGE promove curso técnico de controle para Assessorias de Controle Interno dos órgãos e entidades Estaduais",
    "Uma aeronave da Coordenadoria Integrada de Operações Aéreas (Ciopaer) da Secretaria da Segurança Pública e Defesa Social (SSPDS) realizou o resgate aeromédico de uma jovem de 23 anos vítima de um acidente automobilístico"
]
for sentence in sentences:
    align, spec, stop_tokens, wav = tts(
        model, vocoder_model, sentence, TTS_CONFIG, use_cuda, ap, use_gl=False, trim=True, figures=True
    )